# Create Percentile Scores

Create the Percentile data, from scores cached in the Score notebook. 


In [4]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import rowgenerators as rg
import matplotlib.pyplot as plt
from IPython.display import display 

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()

from demosearch import *
from pathlib import Path

config = ConfigFile.default()
cache = config.cache
rm = RasterManager(config)

In [5]:
# Setup the package and the cache 

pkg = mp.jupyter.open_source_package()
pkg.set_sys_path()
import  pylib 


pkg
 

# Percentile breaks for radius search layers
`civicknowledge.com-percentile-demosearch-1.1.3` Last Update: 2021-06-11T15:28:45

_Percentile breaks for each radius search layer, per CBSA_




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [percentiles](notebooks/Compile.ipynb#pct)**. Percentiles, for scored layers, per CBSA

## References
<ul><li> <strong>scores</string>, <em>metapack+http://s3.amazonaws.com/radius.civicknowledge.com/civicknowledge.com-scores-demosearch.csv#scores</em>. A sample of about 500K scores</li><ul>

In [7]:
#df = pkg.reference('scores').geoframe()
df = rg.dataframe('index:civicknowledge.com-scores-demosearch-2.1.1#scores')
df.drop(columns='geometry',inplace=True)
df.rename(columns={'geoid':'cbsa'}, inplace=True)
df.shape

(12163365, 13)

In [9]:
df.columns

Index(['idx', 'cbsa', 'geometry', 'layer', 'w_r_inv', 'w_r2_inv', 'w_r3_inv',
       'w_circle4', 'w_lin_200', 'w_lin_800', 'w_lin_2000', 'w_half',
       'w_zero'],
      dtype='object')

In [23]:

# All CBSA that have less than 150 records are grouped into a single
# category. 
t = df.groupby(['cbsa','layer']).count().idx.reset_index()
small_cbsa = t.loc[t.idx<150].cbsa.to_list()
df['group_cbsa'] = df.cbsa.mask(df.cbsa.isin(small_cbsa), '31000US00000')
df

,idx,cbsa,geometry,layer,w_r_inv,w_r2_inv,w_r3_inv,w_circle4,w_lin_200,w_lin_800,w_lin_2000,w_half,w_zero,group_cbsa
0,1334420,31000US10100,POINT (-98.453806 45.440505),osmpoints/active,0.622906,0.035156,0.002070,0,0.000000,0.000000,0.000000,10.095112,8.190224,31000US00000
1,1334420,31000US10100,POINT (-98.453806 45.440505),osmpoints/amenity,6.334270,0.524081,0.110543,3,0.000000,0.734973,2.347776,104.281704,81.563407,31000US00000
2,1334420,31000US10100,POINT (-98.453806 45.440505),osmpoints/bar,0.164996,0.009370,0.000546,0,0.000000,0.000000,0.000000,2.599338,2.198676,31000US00000
3,1334420,31000US10100,POINT (-98.453806 45.440505),osmpoints/cafe,0.102979,0.005524,0.000307,0,0.000000,0.000000,0.000000,1.713356,1.426711,31000US00000
4,1334420,31000US10100,POINT (-98.453806 45.440505),osmpoints/casual,0.808071,0.041909,0.002317,0,0.000000,0.000000,0.000000,14.191308,11.382616,31000US00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12163360,1498249,31000US49820,POINT (-99.25775 26.91741),tracts/pop/commute_25lt,323.411327,92.730365,52.267300,970,10.670802,228.675794,683.503661,2355.396242,2001.792483,31000US00000
12163361,1498249,31000US49820,POINT (-99.25775 26.91741),tracts/pop/manufacturing_ocupations,49.026031,13.364911,7.354599,144,1.482056,33.113708,101.265158,381.363643,319.727285,31000US00000
12163362,1498249,31000US49820,POINT (-99.25775 26.91741),tracts/pop/population_18lt,336.511078,93.760830,52.448921,984,10.670802,230.432901,693.577169,2593.956411,2165.912823,31000US00000
12163363,1498249,31000US49820,POINT (-99.25775 26.91741),tracts/pop/seniors,112.803196,25.575009,13.441089,275,2.667700,60.691069,194.056683,1157.842961,914.685922,31000US00000


In [53]:
t = df
t = t.set_index(['idx','cbsa','group_cbsa','layer']).stack().to_frame('score')#.unstack(-2).reset_index()
t = t.reset_index().rename(columns={'level_4':'weight'})
dfs = t.pivot(columns='layer',values='score', index=['idx','cbsa','group_cbsa','weight']).reset_index()


In [54]:
dfs

layer,idx,cbsa,group_cbsa,weight,osmpoints/active,osmpoints/amenity,osmpoints/bar,osmpoints/cafe,osmpoints/casual,osmpoints/entertain,...,tracts/hh/housing_occupied,tracts/hh/housing_owned,tracts/hh/housing_rented_college,tracts/pop/agg_income,tracts/pop/commute_25gte,tracts/pop/commute_25lt,tracts/pop/manufacturing_ocupations,tracts/pop/population_18lt,tracts/pop/seniors,tracts/pop/total_population
0,0,31000US37980,31000US37980,w_circle4,1.000000,7.000000,0.000000,0.000000,NaN,0.000000,...,2424.000000,1413.000000,344.000000,1.938017e+08,NaN,1786.000000,235.000000,1099.000000,813.000000,5976.000000
1,0,31000US37980,31000US37980,w_half,11.577344,130.774783,1.198382,1.555712,NaN,20.606792,...,50608.771332,33336.973356,5105.483243,4.513478e+09,NaN,40945.699314,6637.218922,29707.431311,18114.890884,138781.275012
2,0,31000US37980,31000US37980,w_lin_200,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,...,10.077980,6.817457,1.482056,9.455211e+05,NaN,8.003101,0.889233,4.446167,3.556934,26.677005
3,0,31000US37980,31000US37980,w_lin_2000,0.472580,4.518421,0.000000,0.000000,NaN,0.280325,...,1736.485071,1012.358861,249.316726,1.403702e+08,NaN,1288.111960,173.994015,806.223846,580.489499,4310.758951
4,0,31000US37980,31000US37980,w_lin_800,0.000000,0.734973,0.000000,0.000000,NaN,0.000000,...,303.822825,199.124352,45.732245,2.703787e+07,NaN,236.380359,28.653257,136.206230,105.029266,785.517085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4475371,2249702,31000US47900,31000US47900,w_lin_800,0.990506,4.003146,0.000000,0.000000,0.000000,0.000000,...,NaN,299.599571,45.593093,4.866545e+07,321.246536,143.391523,6.037921,226.173584,306.018901,1171.187767
4475372,2249702,31000US47900,31000US47900,w_r2_inv,0.510503,4.551377,0.066159,0.055762,0.187270,0.364648,...,NaN,262.897014,84.705070,5.450360e+07,398.857995,199.925326,22.169573,256.865055,223.840000,1293.812222
4475373,2249702,31000US47900,31000US47900,w_r3_inv,0.114033,1.411863,0.003063,0.001497,0.007456,0.018888,...,NaN,77.601738,12.671139,1.306981e+07,86.207073,37.646930,1.390532,61.855685,78.913125,305.871830
4475374,2249702,31000US47900,31000US47900,w_r_inv,8.549290,81.901169,1.939729,2.274293,6.521067,11.264600,...,NaN,4610.468586,2476.114049,1.199262e+09,8873.533427,4801.962138,645.059493,5346.017146,3313.278586,27183.123367


In [63]:
first_score_col = dfs.columns[4]
dfs.loc[:, first_score_col:]

layer,osmpoints/active,osmpoints/amenity,osmpoints/bar,osmpoints/cafe,osmpoints/casual,osmpoints/entertain,osmpoints/food,osmpoints/playpark,osmpoints/restaurant,osmpoints/shop,...,tracts/hh/housing_occupied,tracts/hh/housing_owned,tracts/hh/housing_rented_college,tracts/pop/agg_income,tracts/pop/commute_25gte,tracts/pop/commute_25lt,tracts/pop/manufacturing_ocupations,tracts/pop/population_18lt,tracts/pop/seniors,tracts/pop/total_population
0,1.000000,7.000000,0.000000,0.000000,NaN,0.000000,2.000000,0.000000,0.000000,5.000000,...,2424.000000,1413.000000,344.000000,1.938017e+08,NaN,1786.000000,235.000000,1099.000000,813.000000,5976.000000
1,11.577344,130.774783,1.198382,1.555712,NaN,20.606792,11.336516,4.743708,18.377041,79.748279,...,50608.771332,33336.973356,5105.483243,4.513478e+09,NaN,40945.699314,6637.218922,29707.431311,18114.890884,138781.275012
2,0.000000,0.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,...,10.077980,6.817457,1.482056,9.455211e+05,NaN,8.003101,0.889233,4.446167,3.556934,26.677005
3,0.472580,4.518421,0.000000,0.000000,NaN,0.280325,0.945160,0.000000,0.280325,3.012611,...,1736.485071,1012.358861,249.316726,1.403702e+08,NaN,1288.111960,173.994015,806.223846,580.489499,4310.758951
4,0.000000,0.734973,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.734973,...,303.822825,199.124352,45.732245,2.703787e+07,NaN,236.380359,28.653257,136.206230,105.029266,785.517085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4475371,0.990506,4.003146,0.000000,0.000000,0.000000,0.000000,0.000000,0.990506,0.000000,1.981012,...,NaN,299.599571,45.593093,4.866545e+07,321.246536,143.391523,6.037921,226.173584,306.018901,1171.187767
4475372,0.510503,4.551377,0.066159,0.055762,0.187270,0.364648,0.138041,0.416800,0.253965,1.162847,...,NaN,262.897014,84.705070,5.450360e+07,398.857995,199.925326,22.169573,256.865055,223.840000,1293.812222
4475373,0.114033,1.411863,0.003063,0.001497,0.007456,0.018888,0.006325,0.111298,0.014575,0.242344,...,NaN,77.601738,12.671139,1.306981e+07,86.207073,37.646930,1.390532,61.855685,78.913125,305.871830
4475374,8.549290,81.901169,1.939729,2.274293,6.521067,11.264600,4.297376,4.742111,7.581576,18.772347,...,NaN,4610.468586,2476.114049,1.199262e+09,8873.533427,4801.962138,645.059493,5346.017146,3313.278586,27183.123367


In [64]:
dfs.columns[4:]

Index(['osmpoints/active', 'osmpoints/amenity', 'osmpoints/bar',
       'osmpoints/cafe', 'osmpoints/casual', 'osmpoints/entertain',
       'osmpoints/food', 'osmpoints/playpark', 'osmpoints/restaurant',
       'osmpoints/shop', 'osmpoints/travel', 'roads/primary', 'roads/tertiary',
       'tracts/hh/households', 'tracts/hh/households_cohabiting',
       'tracts/hh/households_unmaried', 'tracts/hh/housing_occupied',
       'tracts/hh/housing_owned', 'tracts/hh/housing_rented_college',
       'tracts/pop/agg_income', 'tracts/pop/commute_25gte',
       'tracts/pop/commute_25lt', 'tracts/pop/manufacturing_ocupations',
       'tracts/pop/population_18lt', 'tracts/pop/seniors',
       'tracts/pop/total_population'],
      dtype='object', name='layer')

In [56]:
osm_cols = [ c.rsplit('/',1)[-1] for c in dfs.columns if c.startswith('osmpoints')]
road_cols = [ c.rsplit('/',1)[-1] for c in dfs.columns if c.startswith('roads')]
hh_cols = [ c.rsplit('/',1)[-1] for c in dfs.columns if c.startswith('tracts_hh')]
pop_cols = [ c.rsplit('/',1)[-1] for c in dfs.columns if c.startswith('tracts_pop')]
demo_cols = hh_cols + pop_cols
osm_cols           

['active',
 'amenity',
 'bar',
 'cafe',
 'casual',
 'entertain',
 'food',
 'playpark',
 'restaurant',
 'shop',
 'travel']

In [68]:
# Create percentiles dataset

def make_pctile_df(df):
    """Create the percentiles dataset"""

    frames = []
    for idx, g in df.groupby(['group_cbsa','weight']):
        t = np.nanpercentile(g.loc[:, first_score_col:], np.linspace(0,100,101), axis=0)
        pct = pd.DataFrame(t, columns = dfs.columns[4:]).round(2)
        pct[['cbsa','weight']] = idx
        pct.index.name = 'pct'
        frames.append(pct.set_index(['cbsa','weight'],append=True))

    pct = pd.concat(frames) 
    
    return pct
 
def col_pctile(df, cbsa, col, value):
    """Use the CBSA percentiles map to find the percentile of a column value """
    t = df.loc[(slice(None), cbsa),:]
    # The sorting ensures that when there ae a lot of percentiles that have value 0, we
    # return the largest percentile value, rather than 0
    return (t[col]-value).abs().sort_values(ascending=False).idxmin()[0]
    
pct = make_pctile_df(dfs)
    
#v = pct.sort_index().loc[ (50, '31000US31080'), :].osmpoints_cafe
#col_pctile(pct, '31000US31080', 'osmpoints_cafe', v) # SHould be 50
    

/Users/eric/opt/anaconda3/envs/radius38/lib/python3.8/site-packages/numpy/lib/nanfunctions.py:1389: RuntimeWarning: All-NaN slice encountered
  result = np.apply_along_axis(_nanquantile_1d, axis, a, q,


In [69]:
pct


,,layer,osmpoints/active,osmpoints/amenity,osmpoints/bar,osmpoints/cafe,osmpoints/casual,osmpoints/entertain,osmpoints/food,osmpoints/playpark,osmpoints/restaurant,osmpoints/shop,...,tracts/hh/housing_occupied,tracts/hh/housing_owned,tracts/hh/housing_rented_college,tracts/pop/agg_income,tracts/pop/commute_25gte,tracts/pop/commute_25lt,tracts/pop/manufacturing_ocupations,tracts/pop/population_18lt,tracts/pop/seniors,tracts/pop/total_population
pct,cbsa,weight,,,,,,,,,,,,,,,,,,,,,
0,31000US00000,w_circle4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.00,0.00,0.000000e+00,0.00,0.00,0.00,0.00,0.00,0.00
1,31000US00000,w_circle4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,3.0,1.00,0.00,2.676810e+05,0.00,1.00,0.00,1.00,1.00,7.00
2,31000US00000,w_circle4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,4.0,3.00,0.00,4.426663e+05,1.00,3.00,0.00,1.00,1.00,14.00
3,31000US00000,w_circle4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,7.0,4.00,0.00,6.001740e+05,1.00,4.00,0.00,3.00,3.00,19.00
4,31000US00000,w_circle4,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,9.0,6.00,0.00,7.186090e+05,3.00,4.00,0.00,4.00,4.00,25.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,31000US49700,w_zero,10.90,73.61,0.94,0.94,9.54,7.38,6.64,10.90,5.62,29.81,...,NaN,11423.27,1188.35,1.586180e+09,9545.08,15036.72,2877.22,17110.89,8792.23,64844.20
97,31000US49700,w_zero,10.93,73.69,0.94,0.94,9.58,7.39,6.73,10.93,5.65,29.94,...,NaN,11437.75,1192.08,1.589973e+09,9594.19,15071.38,2885.66,17154.80,8798.56,64927.00
98,31000US49700,w_zero,10.99,73.78,0.94,0.97,9.64,7.41,6.75,10.99,5.67,30.01,...,NaN,11468.09,1195.78,1.593708e+09,9640.64,15123.76,2902.10,17226.85,8813.65,65066.17


In [82]:
t =pct.reset_index()
t[t.weight=='w_zero']['tracts/pop/total_population'].describe()

count    4.343000e+04
mean     3.664259e+04
std      5.635989e+04
min      2.431000e+01
25%      7.467998e+03
50%      1.953457e+04
75%      4.584773e+04
max      1.592808e+06
Name: tracts/pop/total_population, dtype: float64

In [87]:
pct[['tracts/pop/total_population']].unstack().describe()

layer  tracts/pop/total_population                                             \
weight                   w_circle4        w_half     w_lin_200     w_lin_2000   
count                 43430.000000  4.343000e+04  43430.000000   43430.000000   
mean                   3046.992931  6.376507e+04     17.626217    2217.750121   
std                    4601.424289  1.030531e+05     33.210950    3312.470905   
min                       0.000000  4.116000e+01      0.000000       0.000000   
25%                     347.000000  1.355361e+04      1.480000     257.950000   
50%                    1651.590000  3.179651e+04      7.990000    1219.965000   
75%                    4233.930000  7.680407e+04     23.710000    3086.360000   
max                  178772.000000  2.981038e+06   2169.430000  120954.960000   

layer                                                                          
weight     w_lin_800      w_r2_inv      w_r3_inv        w_r_inv        w_zero  
count   43430.000000  43430.000000  43430.000000   43430.000000  4.343000e+04  
mean      472.238774    282.781687    100.361619    3136.032343  3.664259e+04  
std       774.853894    431.651525    173.937634    4747.983182  5.635989e+04  
min         0.000000      0.060000      0.000000       1.970000  2.431000e+01  
25%        46.950000     40.502500      9.862500     648.155000  7.467998e+03  
50%       235.155000    157.005000     48.190000    1739.620000  1.953457e+04  
75%       650.067500    386.810000    136.825000    3987.475000  4.584773e+04  
max     39475.460000  19317.520000   9865.940000  144775.460000  1.592808e+06

In [37]:
pct = pct.sort_index(level='cbsa')

In [39]:
pct  = pct.reset_index()
pct.head()

,pct,cbsa,tracts_hh_households,tracts_hh_households_cohabiting,tracts_hh_households_unmaried,tracts_hh_housing_occupied,tracts_hh_housing_owned,tracts_hh_housing_rented_college,tracts_pop_commute_25gte,tracts_pop_commute_25lt,...,osmpoints_amenity,osmpoints_bar,osmpoints_cafe,osmpoints_casual,osmpoints_entertain,osmpoints_food,osmpoints_playpark,osmpoints_restaurant,osmpoints_shop,osmpoints_travel
0,0,31000US00000,1.78,1.67,0.00,1.90,0.00,0.00,0.67,0.94,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00
1,1,31000US00000,87.07,83.76,2.42,89.07,58.86,1.75,32.43,44.82,...,0.77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.14
2,2,31000US00000,124.27,118.65,4.84,123.36,84.20,2.82,42.61,76.25,...,1.48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,1.82
3,3,31000US00000,170.04,162.25,6.65,170.73,116.80,3.88,63.79,106.72,...,2.31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,3.22
4,4,31000US00000,239.98,228.10,8.32,241.71,165.05,4.97,74.91,123.28,...,3.38,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.34,4.47
